In [42]:
import argparse
from IPython import embed
import os.path as osp
import time

import numpy as np
import networkx as nx
import scipy as sp

import torch
import torch.nn as nn
from torch.distributions import Normal
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, ChebConv  # noqa

In [4]:
dataset = 'Cora'
path = path = '../data/geometric/CORA'
dataset = Planetoid(path, dataset, T.NormalizeFeatures())
data = dataset[0]

In [92]:
# class GCN(nn.Module):
#     def __init__(self):
#         super(GCN, self).__init__()
#         self.conv1 = GCNConv(dataset.num_features, 16, improved=False)
#         self.conv2 = GCNConv(16, dataset.num_classes, improved=False)

#     def forward(self):
#         x, edge_index = data.x, data.edge_index
#         x = F.relu(self.conv1(x, edge_index))
#         x = F.dropout(x, training=self.training)
#         x = self.conv2(x, edge_index)
        
#         return F.log_softmax(x, dim=1)
    
# class Encoder(nn.Module):
#     def __init__(self, n_feat, z_dim, hidden_dim):
#         super(Encoder, self).__init__()
#         self.conv1 = GCNConv(n_feat, 16, improved=False)
#         self.conv2 = GCNConv(16, dataset.num_classes, improved=False)
        
#         self.gc1 = GCNConv(n_feat, hidden_dim)
#         self.gc2_mu = GCNConv(hidden_dim, z_dim)
#         self.gc2_sig = GCNConv(hidden_dim, z_dim)
        
# #         Setup for non-linearities
#         self.softplus = nn.Softplus()
#         self.relu = nn.ReLU()


#     def forward(self):
# #         x, edge_index = data.x, data.edge_index
# #         x = F.relu(self.conv1(x, edge_index))
# #         x = F.dropout(x, training=self.training)
# #         x = self.conv2(x, edge_index)
        
# #         return F.log_softmax(x, dim=1)

#         hidden = self.softplus(self.gc1(x, adj))
#         z_loc = self.gc2_mu(hidden, adj)
#         z_scale = torch.exp(self.gc2_sig(hidden, adj))

#         return z_loc, z_scale

    
# class Decoder(nn.Module):
#     def __init__(self, z_dim, hidden_dim):
#         super(Decoder, self).__init__()
#         # setup the two linear transformations used for the decoder 
#         self.fc1 = nn.Linear(z_dim, hidden_dim)
#         self.fc21 = nn.Linear(hidden_dim, 10)
#         # setup the non-linearities
#         self.softplus = nn.Softplus()

#     def forward(self, z):
#         # define the forward computation on the latent z
#         # first compute the hidden units

#         hidden = self.softplus(self.fc1(z))
# #         loc_img = torch.sigmoid(self.fc21(hidden))
# #         return loc_img

#         return F.log_softmax(self.fc21(hidden))
    
# class VAE(nn.Module):
#     def __init__(self, z_dim=16, hidden_dim=400, use_cuda=False):
#         super(VAE, self).__init__()

#         self.encoder = Encoder(dataset.num_features, z_dim, hidden_dim)
#         self.decoder = Decoder(z_dim, hidden_dim)

#         if use_cuda:
#             self.cuda()
            
#         self.use_cuda = use_cuda
#         self.z_dim = z_dim
        
#     def forward(self):
#         z_loc, z_scale = self.encoder.forward()
#         z = Normal(z_loc, z_scale)
        
#         out = self.decoder.forward(z)
#         return out

In [169]:
class VAE(nn.Module):
    def __init__(self, num_features, z_dim, hidden_dim, dropout, use_cuda=False):
        super(VAE, self).__init__()
        self.conv1 = GCNConv(num_features, z_dim, improved=False)
        self.conv2 = GCNConv(z_dim, hidden_dim, improved=False)
        self.decoder = InnerProductDecoder(dropout, activation=lambda x:x)
        
#         self.gc1 = GCNConv(n_feat, hidden_dim)
#         self.gc2_mu = GCNConv(hidden_dim, z_dim)
#         self.gc2_sig = GCNConv(hidden_dim, z_dim)

    def encode(self, x, adj):
        hidden = self.conv1(x, adj)
        return self.conv2(hidden, adj)
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(logvar)
            epsilon = torch.rand_like(mu)
        else:
            return mu
        
    def forward(self, x, adj):
        mu, logvar = self.enode(x, adj)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar
            

class InnerProductDecoder(nn.Module):
    """Decoder for using inner product for prediction.
       https://github.com/zfjsail/gae-pytorch/blob/master/gae/model.py"""

    def __init__(self, dropout, activation=torch.sigmoid):
        super(InnerProductDecoder, self).__init__()
        self.dropout = dropout
        self.activation = activation

    def forward(self, z):
        z = F.dropout(z, self.dropout, training=self.training)
        adj = self.act(torch.mm(z, z.t()))
        return adj

In [170]:
def loss_function(preds, labels, mu, logvar, n_nodes, norm, pos_weight):
    cost = norm * F.binary_cross_entropy_with_logits(preds, labels, pos_weight=pos_weight)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 / n_nodes * torch.mean(torch.sum(
        1 + 2 * logvar - mu.pow(2) - logvar.exp().pow(2), 1))
    return cost + KLD

In [171]:

# Get adjacency matrix and various edge lists 

features = torch.FloatTensor(np.array(data['x']))

n_nodes, feat_dim = features.shape

edgeList = np.array(data['edge_index'].transpose(1,0 ))
adj = nx.adjacency_matrix(nx.from_edgelist(edgeList))

print('Shape of adjacency matrix: ', adj.shape)
print('Length of features: ', len(features))

def sparse_to_tuple(sparse_mx):
    sparse_mx = sparse_mx.tocoo()
        
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

adj = adj - sp.sparse.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape =adj.shape)
adj.eliminate_zeros()

assert np.diag(adj.todense()).sum() == 0

adj_triu = sp.sparse.triu(adj)
adj_tuple = sparse_to_tuple(adj_triu)

edges = adj_tuple[0]
edges_all = sparse_to_tuple(adj)[0]
all_edge_idx = list(range(edges.shape[0]))

adj_train = adj[data['train_mask']]

train_edges = edgeList 

train_mask_indices = [i for i, x in enumerate(data['train_mask']) if x]
valid_mask_indices = [i for i, x in enumerate(data['val_mask']) if x]
test_mask_indices = [i for i, x in enumerate(data['test_mask']) if x]

train_edges = adj_train[train_mask_indices]
valid_edges = adj_train[valid_mask_indices]
test_edges = adj_train[test_mask_indices]

Shape of adjacency matrix:  (2708, 2708)
Length of features:  2708


In [ ]:
# Preprocess graph

def preprocess_graph(adj):
    adj = sp.sparse.coo_matrix(adj)
    adj_ = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = sp.sparse.diags(np.power(rowsum, -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
    # return sparse_to_tuple(adj_normalized)
    return sparse_mx_to_torch_sparse_tensor(adj_normalized)

adj_norm = preprocess_graph(adj)
adj_label = adj_train + sp.eye(adj_train.shape[0])
adj_label = torch.FloatTensor(adj_label.toarray())
pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()

norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)
hidden_emb = None 


model = VAE(data.num_features, 32, 16, dropout=0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = VAE().to(device), data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


In [ ]:
for epoch in range(2):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    recovered, mu, logvar = model(features, adjnorm)
    loss = loss_function(recovered, adj_label, mu, logvar, n_nodes, norm, pos_weight)
    
    loss.backward()
    cur_loss = loss.item()
    optimizer.step()
    
    hidden_emb = mu.data.numpy()
    roc_curr, ap_curr = get_roc_score(hidden_emb, adj_orig, val_edges, val_edges_false)
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cur_loss),
          "val_ap=", "{:.5f}".format(ap_curr),
          "time=", "{:.5f}".format(time.time() - t)
          )

print("Optimization Finished!")
roc_score, ap_score = get_roc_score(hidden_emb, adj_orig, test_edges, test_edges_false)
print('Test ROC score: ' + str(roc_score))
print('Test AP score: ' + str(ap_score))

In [94]:
# def train():
#     model.train()
#     optimizer.zero_grad()
#     F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
#     optimizer.step()


# def test():
#     model.eval()
#     logits, accs = model(), []
#     for _, mask in data('train_mask', 'val_mask', 'test_mask'):
#         pred = logits[mask].max(1)[1]
#         acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
#         accs.append(acc)
#     return accs

def train():
    model.train()
    optimizer.zero_grad()
    
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()

def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [95]:
best_val_acc = test_acc = 0
for epoch in range(1, 100):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, train_acc, best_val_acc, test_acc))


NameError: name 'x' is not defined

In [89]:
# parser = argparse.ArgumentParser(description="parse args")
# parser.add_argument('-n', '--num-epochs', default=2, type=int, help='number of training epochs')
# parser.add_argument('-tf', '--test-frequency', default=5, type=int, help='how often we evaluate the test set')
# parser.add_argument('-lr', '--learning-rate', default=2.0e-3, type=float, help='learning rate')
    
# parser.add_argument('--cuda', action='store_true', default=False, help='whether to use cuda')
# parser.add_argument('--jit', action='store_true', default=False, help='whether to use PyTorch jit')
# parser.add_argument('-visdom', '--visdom_flag', action="store_true", help='Whether plotting in visdom is desired')
# parser.add_argument('--time', default=int(time.time()), help="Current system time")

# parser.add_argument('--name', default='Mnist', help="Name of the dataset")
# parser.add_argument('--save', default=False, help="Whether to save the trained model")
# args = parser.parse_args(args=[])

In [60]:
# model = main(args)

In [76]:
print(data.train_mask)
print(data.y[data.train_mask])

tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8)
tensor([3, 4, 4, 0, 3, 2, 0, 3, 3, 2, 0, 0, 4, 3, 3, 3, 2, 3, 1, 3, 5, 3, 4, 6,
        3, 3, 6, 3, 2, 4, 3, 6, 0, 4, 2, 0, 1, 5, 4, 4, 3, 6, 6, 4, 3, 3, 2, 5,
        3, 4, 5, 3, 0, 2, 1, 4, 6, 3, 2, 2, 0, 0, 0, 4, 2, 0, 4, 5, 2, 6, 5, 2,
        2, 2, 0, 4, 5, 6, 4, 0, 0, 0, 4, 2, 4, 1, 4, 6, 0, 4, 2, 4, 6, 6, 0, 0,
        6, 5, 0, 6, 0, 2, 1, 1, 1, 2, 6, 5, 6, 1, 2, 2, 1, 5, 5, 5, 6, 5, 6, 5,
        5, 1, 6, 6, 1, 5, 1, 6, 5, 5, 5, 1, 5, 1, 1, 1, 1, 1, 1, 1])
